<a href="https://colab.research.google.com/github/yujung-04/3-2-/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9A%94%EC%95%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate sentencepiece summa evaluate rouge-score -q



In [ ]:
import pandas as pd
import re
import torch

from datasets import Dataset, DatasetDict
from summa.summarizer import summarize

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [ ]:
train_path = "/content/train_clean.csv"
val_path   = "/content/valtest_clean.csv"
train_df = pd.read_csv(train_path)
val_df   = pd.read_csv(val_path)

print("train:", train_df.shape)
print("val  :", val_df.shape)
train_df.head()


train: (900, 5)
val  : (226, 5)


,title,content,category,agree_count,url
0,지귀연 판사 탄핵에 관한 청원,우리는 친위쿠데타 군사반란 우두머리를 제멋대로 석방한 법술사 지귀연을 민주공화국 대...,정치/선거/국회운영,58209,https://petitions.assembly.go.kr/proceed/close...
1,「아동·청소년의 성보호에 관한 법률」 상 가상의 표현물 제외에 관한 청원,[배경 및 문제 인식] 1. 가상의 표현물 단속에 투입되는 과도한 예산 가상의 아동...,저출산/고령화/아동/청소년/가족,18406,https://petitions.assembly.go.kr/proceed/close...
2,극희귀난치성 질환 환자를 위한 ‘리브말리액’ 건강보험 신속한 적용 요청에 관한 청원,"1. 가려움증과 피부손상은 아이의 일상을 파괴합니다 밤마다 긁어 생긴 상처, 그 위...",보건의료,2743,https://petitions.assembly.go.kr/proceed/close...
3,조희대 대법관을 포함한 10명의 대법관과 지귀연 판사 즉각 탄핵에 관한 청원,조희대 대법원장을 포함한 10명의 대법관들은 절차와 규정을 어기며 야권의 유력 대선...,정치/선거/국회운영,64161,https://petitions.assembly.go.kr/proceed/close...
4,보건복지부 출신 관료들의 관련 기업 재취업 금지법 제정 요구에 관한 청원,금번 의료사태를 국민들이 겪으며 1년이 넘는 기간 동안 큰 고통속에 살았음을 우리는...,행정/지방자치,9637,https://petitions.assembly.go.kr/proceed/close...


In [ ]:
def generate_summary(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    summary_ids = model.generate(
        **inputs,
        max_length=150,
        min_length=30,
        no_repeat_ngram_size=3,
        repetition_penalty=1.8,
        length_penalty=1.0,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
from summa.summarizer import summarize
import re

def make_summary(text):
    text = str(text).strip()
    text = re.sub(r'\s+', ' ', text)

    # TextRank 적용
    try:
        s = summarize(text, ratio=0.08)
    except:
        s = ""

    # 비어있으면 전체에서 가져오기
    if not s or len(s.strip()) < 20:
        s = text

    # 문장 나누기
    sentences = re.split(r'(?<=[.!?])\s+', s)
    if len(sentences) < 2:
        sentences = s.split('.')

    clean_sentences = []
    for sent in sentences:
        sent = sent.strip()
        # 끊긴 문장 제거
        if len(sent) < 20:
            continue
        # 알파벳/한글 비율 너무 낮으면 제거
        letters = sum(c.isalpha() for c in sent)
        if letters / max(len(sent),1) < 0.3:
            continue

        # 문장 끝이 이상하면 붙여주기
        if sent[-1] not in ['.', '!', '?', '다', '요', '함', '음']:
            sent += "."

        clean_sentences.append(sent)

    # 적당한 문장 1~2개만 반환
    if len(clean_sentences) >= 2:
        summary = " ".join(clean_sentences[:2])
    elif len(clean_sentences) == 1:
        summary = clean_sentences[0]
    else:
        summary = text[:150] + "..."

    return summary


In [ ]:
from summa.summarizer import summarize
import re

def make_summary_focus(text):
    text = str(text).strip()
    text = re.sub(r'\s+', ' ', text)

    # 너무 짧은 본문은 그냥 그대로 반환
    if len(text) < 120:
        return text

    # 1) 문장 단위로 분리
    sentences = re.split(r'(?<=[.!?？。])\s+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 0]

    if len(sentences) == 0:
        return text

    # 2) 키워드/패턴들 정의 (요청/결론/해결/법안 위주)
    request_keywords = [
        "요청", "촉구", "해 주십시오", "해주십시오",
        "해 주시기 바랍니다", "해주시기 바랍니다",
        "주시기 바랍니다", "바랍니다", "간곡히 청원드립니다",
        "간절히 바랍니다", "해결해 주십시오", "청원합니다",
        "재고해 주십시오", "통과시켜 주십시오", "추진해 주십시오",
        "신속히 추진", "강력히 촉구"
    ]
    solution_keywords = [
        "제도", "개선", "지원", "도입", "강화", "구축",
        "정비", "보완", "보호", "체계", "정책", "법률",
        "입법", "제정", "특별법", "대책", "방안", "시행",
        "수립", "정착", "개편"
    ]
    problem_keywords = [
        "문제가 있습니다", "심각한", "위험", "피해", "고통",
        "혼란", "부당", "차별", "위협", "불합리", "위기", "붕괴"
    ]
    law_keywords = [
        "법안", "법률안", "일부개정법률안", "특별법",
        "폐지", "개정안", "입법", "제정"
    ]

    conclusion_starters = [
        "따라서", "이에 따라", "그러므로", "이러한 이유로",
        "결론적으로", "이와 같은 이유로", "이러한 상황에서",
        "이에 대해", "이상과 같은 제도적 개선을 통해",
        "이상과 같은 이유로"
    ]

    best_sent = None
    best_score = -1.0
    best_idx = -1
    total_sents = len(sentences)

    for idx, sent in enumerate(sentences):
        s = sent.strip()

        # 1) 너무 짧은 문장 제거 (메모/조각 느낌)
        if len(s) < 25:
            continue

        # 2) 앞에 번호 달린 리스트(1. ~, 2) ~) 제거
        if re.match(r'^\d+\s*[\.\)]\s*', s):
            continue

        # 3) 문자 비율 너무 낮으면(숫자/기호 덩어리) 제거
        letters = sum(ch.isalpha() for ch in s)
        if letters / max(len(s), 1) < 0.3:
            continue

        score = 0.0

        # 요청/촉구/청원 문장
        if any(kw in s for kw in request_keywords):
            score += 4.0

        # 제도·법·지원·대책 문장
        if any(kw in s for kw in solution_keywords):
            score += 2.5

        # 법안/입법 관련 문장
        if any(kw in s for kw in law_keywords):
            score += 2.0

        #  문제의식 강조 문장
        if any(kw in s for kw in problem_keywords):
            score += 1.5

        #  따라서/이에 따라/결론적으로… 같은 결론 문장
        if any(s.startswith(st) for st in conclusion_starters):
            score += 2.0

        # 청원 + 합니다/드립니다 조합이면 더 가중치
        if "청원" in s and ("합니다" in s or "드립니다" in s):
            score += 2.0

        # 위치 보정: 도입부/결론부 가산점
        if idx == 0:
            score += 0.5          # 문제 제기 도입부
        if idx >= total_sents - 3:
            score += 1.0          # 마지막 3문장 → 결론/요청일 확률 ↑

        # 숫자 나열이 너무 많은 문장은 살짝 패널티
        digit_count = sum(ch.isdigit() for ch in s)
        if digit_count >= 6 and not any(kw in s for kw in ("법", "법안", "특별법", "청원")):
            score -= 1.0

        if score > best_score:
            best_score = score
            best_sent = s
            best_idx = idx

    # 8) 그래도 못 찾으면 TextRank 또는 앞부분 fallback
    if best_sent is None:
        try:
            s = summarize(text, ratio=0.1)
            if s and len(s.strip()) > 0:
                best_sent = s.strip()
            else:
                best_sent = text[:200]
        except:
            best_sent = text[:200]
        return best_sent

    # 9) 선택된 문장이 애매하게 끊긴 느낌이면, 바로 다음 문장 한 개를 붙여서 보완
    #    (예: 숫자 나열 끝, 조사로 끝나는 경우 등)
    def looks_cut_off(sent):
        sent = sent.strip()
        # 문장 끝이 너무 애매한 경우
        if sent[-1] in ",;:":
            return True
        # 한국어 종결 어미 없이 숫자/영어로 끝나는 경우
        if re.search(r'[0-9a-zA-Z]$', sent):
            return True
        # 종결 어미(다/니다/요/바랍니다/청원합니다/촉구합니다 등)로 끝나면 괜찮음
        if re.search(r'(다|니다|요|바랍니다|청원합니다|촉구합니다)[\.\!]?$', sent):
            return False
        # 너무 짧지 않은데 마침표/느낌표/물음표도 없으면 애매한 걸로 판단
        if len(sent) > 40 and sent[-1] not in ".!?":
            return True
        return False

    if looks_cut_off(best_sent) and 0 <= best_idx < total_sents - 1:
        next_sent = sentences[best_idx + 1].strip()
        # 다음 문장도 너무 짧거나 리스트형이면 무시
        if len(next_sent) > 20 and not re.match(r'^\d+\s*[\.\)]\s*', next_sent):
            best_sent = best_sent.rstrip(" ,;:") + " " + next_sent

    # 마지막으로, 문장 끝 마침표 보정
    if best_sent[-1] not in ".!?":
        best_sent = best_sent + "."

    return best_sent


In [ ]:
train_df["summary"] = train_df["content"].apply(make_summary_focus)
val_df["summary"]   = val_df["content"].apply(make_summary_focus)


In [ ]:
train_df[["content", "summary"]].head(10)

,content,summary
0,우리는 친위쿠데타 군사반란 우두머리를 제멋대로 석방한 법술사 지귀연을 민주공화국 대...,박정희 독재의 1975년 인혁당 사법살인부터 2004년 헌법재판소의 수도이전 관습헌...
1,"1. 가려움증과 피부손상은 아이의 일상을 파괴합니다 밤마다 긁어 생긴 상처, 그 위...",부디 보건당국과 정부가 이 아이들의 손을 잡아주시기를 간절히 요청드립니다.
2,조희대 대법원장을 포함한 10명의 대법관들은 절차와 규정을 어기며 야권의 유력 대선...,"직권남용, 직무유기, 공무원의 정치중립의무 위반 및 공직선거법 위반 등의 죄를 범한..."
3,금번 의료사태를 국민들이 겪으며 1년이 넘는 기간 동안 큰 고통속에 살았음을 우리는...,따라서 보건복지부 관료들의 퇴직후 관련 기업에 재취업을 금지하는 법안을 통해 공무원...
4,"동물병원 진료 중 발생하는 문제에 대해 보호자는 정보를 충분히 제공받지 못하고, 진...",청원의 주요 요청 사항 다음과 같은 수의사법 개정 및 제도 도입을 요청합니다 1.
5,제가 보호하고 있는 아이는 입양특례법상 아동으로 전제위탁 절차를 통해 저희 가족과 ...,부디 국회가 본 청원을 바탕으로 입양 제도 전반을 재검토해주시길 간곡히 요청드립니다.
6,정부와 한국은행이 추진하고 있는 CBDC(중앙은행 디지털화폐) 및 디지털 바우처 시...,이에 국민적 공감대 형성과 신중한 제도 설계가 선행되어야 함을 강조하며 CBDC 도...
7,수고많으십니다. 비덴트 주주로서 말씀드리고자 합니다. 한국거래소 코스닥시장위원회(1...,"무작정 내리는 상장폐지가 답이 아니라, 투자자 보호 원칙에 따라 모두가 살수 있는 ..."
8,"1. OOO OOO의 으뜸기업 선정 관련, 국감에서 선정 결정 과정에 대한 기록물이...","OOO의 과로사 문제에서, 본청원의 제기 시점까지 정혜경 의원이 요구한 자료 미제출..."
9,안녕하세요. 저희 사하구 하단동 및 사상구 엄궁동 주민들은 (주)삼정환경산업의 이전...,"주민들의 건강과 안전을 위해, 교통공사와 행정기관의 책임 있는 조치와 근본적 대책을..."


In [ ]:
content_len = train_df["content"].str.len()
cut_len = content_len.quantile(0.97)   # 상위 3% 컷

train_df_short = train_df[content_len <= cut_len].reset_index(drop=True)

print("원래 train 개수 :", len(train_df))
print("필터 후 train 개수:", len(train_df_short))


원래 train 개수 : 787
필터 후 train 개수: 763


In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df_short)
val_dataset   = Dataset.from_pandas(val_df)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
})

raw_datasets


DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'category', 'agree_count', 'url', 'summary'],
        num_rows: 763
    })
    validation: Dataset({
        features: ['title', 'content', 'category', 'agree_count', 'url', 'summary'],
        num_rows: 200
    })
})

In [ ]:
model_name = "gogamza/kobart-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["content"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,  # 원래 컬럼(title, content 등) 제거
)

# Filter out examples where labels are empty
# This often happens if the summary was an empty string or became empty after tokenization/truncation
tokenized_datasets = tokenized_datasets.filter(lambda example: len(example["labels"]) > 0)

tokenized_datasets

Map:   0%|          | 0/763 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/763 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 763
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [ ]:
# 1) NaN 제거
train_df = train_df.dropna(subset=["content", "summary"])
val_df   = val_df.dropna(subset=["content", "summary"])

# 2) 공백만 있는 요약 제거
train_df = train_df[train_df["summary"].str.strip() != ""].reset_index(drop=True)
val_df   = val_df[val_df["summary"].str.strip() != ""].reset_index(drop=True)

print("train 정리 후:", train_df.shape)
print("val 정리 후  :", val_df.shape)


train 정리 후: (787, 6)
val 정리 후  : (200, 6)


In [ ]:
max_input_length = 512
max_target_length = 64   # 요약은 한두 문장 정도로

def preprocess_function(examples):
    inputs = examples["content"]
    targets = examples["summary"]

    # 입력
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length",
    )

    # 타겟(요약)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding="max_length",   # 🔹 요약도 길이 0이 안 나오게
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:

tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets


Map:   0%|          | 0/763 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 763
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

training_args = TrainingArguments(
    output_dir="/content/kobart_summary_textrank_ft",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=3e-5,
    warmup_steps=200,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,

)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

trainer.save_model("/content/kobart_summary_textrank_ft")
tokenizer.save_pretrained("/content/kobart_summary_textrank_ft")

print("학습 완료 & 저장: /content/kobart_summary_textrank_ft")


/tmp/ipython-input-3496434532.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.030400
100,0.011700
150,0.027100
200,0.028600
250,0.015000
300,0.019400
350,0.037900
400,0.027700
450,0.031200
500,0.058800


✅ 학습 완료 & 저장: /content/kobart_summary_textrank_ft


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/kobart_summary_textrank_ft"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
model.eval()

def summarize_beam(text, max_len=64):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=40,
            num_beams=4,
            no_repeat_ngram_size=3,
            repetition_penalty=2.0,
            length_penalty=1.0,
            early_stopping=True,
        )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [ ]:

from tqdm import tqdm

records = []

for i in tqdm(range(len(val_df))):
    content = val_df.loc[i, "content"]
    gold = val_df.loc[i, "summary"]
    pred = summarize_beam(content)

    records.append({
        "index": i,
        "content": content,
        "summary_gold": gold,
        "summary_pred": pred
    })

pred_df = pd.DataFrame(records)
pred_df.head()



100%|██████████| 200/200 [02:01<00:00,  1.64it/s]


,index,content,summary_gold,summary_pred
0,0,삼권분립 안에 속해 있는 사법부의 신뢰를 또 한번 무너졌습니다. 이재명 재판에 1심...,정말 이 청원을 올리는 이유는 정치판사가 아닌 누구에게도 공평한 판사를 원하는 겁니다.,이재명 재판에 1심 선고에서 사진은 조작이 아니라는 판결이 있었지만 1심 판사들의 ...
1,1,"이유 윤석열 전 대통령의 장기 구금은 단순한 개인의 문제가 아니라, 대한민국 법치주...","2 교정 당국은 피구속인의 치료 면회 법률대리 접근을 보장하고, 인권 보호를 위한 ...",향후 재판 과정에서는 정치적 고려 없는 공정한 절차와 사법 정의의 회복이 이루어지도...
2,2,"1.현행의 사전투표, 부재자 투표 금지 2. 투표함 이동 금지 투표 후 현장에서 바...","부징선거 관련자 및 투,개표 조작에 관여한 자는 무기 및 사형에 처한다로 관련법을 ...","부징선거 관련자 및 투,개표 조작에 관여한 자는 무기 및 사형에 처한다로 관련법을 ..."
3,3,"보조교사와 연장반 교사는 담임교사와 마찬가지로 아이들을 돌보고, 수업을 준비하고, ...",[청원 내용] 정부와 관계 당국에 다음과 같은 실질적 제도 개선을 요청합니다.,"또한 교사 개인이 행사비용, 학부모 선물비 등을 부담하게 하거나, 회식조차 차별받는..."
4,4,도덕성이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...,도덕성이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...,사상적이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...


In [ ]:

pred_df.to_csv("/content/수정5.csv", index=False)
pred_df.to_excel("/content/수정5.xlsx", index=False)

print("✅ /content/수정5.(csv/xlsx) 저장 완료")


✅ /content/수정5.(csv/xlsx) 저장 완료


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

refs = pred_df["summary_gold"].tolist()
preds = pred_df["summary_pred"].tolist()

scores = rouge.compute(predictions=preds, references=refs)

print("✅ ROUGE (validation, 샘플 {}개)".format(len(pred_df)))
for k, v in scores.items():
    print(f"{k}: {v:.4f}")


✅ ROUGE (validation, 샘플 200개)
rouge1: 0.1287
rouge2: 0.0575
rougeL: 0.1272
rougeLsum: 0.1271


In [ ]:

for i in range(5):
    print(f"=== 예시 {i} ===")
    print("원문 일부     :", pred_df.loc[i, "content"][:200], "...")
    print("정답 요약(gold):", pred_df.loc[i, "summary_gold"])
    print("모델 요약(pred):", pred_df.loc[i, "summary_pred"])
    print()


=== 예시 0 ===
원문 일부     : 삼권분립 안에 속해 있는 사법부의 신뢰를 또 한번 무너졌습니다. 이재명 재판에 1심 선고에서 사진은 조작이 아니라는 판결이 있었지만 1심 판사들의 의견을 무시하고 추가적인 수사없이 사진 확대는 조작이라는 단어로 대한민국의 법의 질서를 어지럽혔습니다. 국민들은 사진 확대로 무죄가 나왔다는 황당한 판결이 무법천지로 살아가도 된다는 뜻이 아니겠습니까? 김문기와  ...
정답 요약(gold): 정말 이 청원을 올리는 이유는 정치판사가 아닌 누구에게도 공평한 판사를 원하는 겁니다.
모델 요약(pred): 이재명 재판에 1심 선고에서 사진은 조작이 아니라는 판결이 있었지만 1심 판사들의 의견을 무시하고 추가적인 수사없이 사진 확대는 조작이라는 단어로 대한민국의 법의 질서를 어지럽혔습니다.ECD 기여할습니까?박스겠습니다. 통합진보당대한민

=== 예시 1 ===
원문 일부     : 이유 윤석열 전 대통령의 장기 구금은 단순한 개인의 문제가 아니라, 대한민국 법치주의의 근간을 흔드는 중대한 사안입니다. 선구속 후재판 이라는 전례 없는 조치는 법이 정치의 도구로 전락하는 위험을 보여주고 있습니다. 헌법이 보장하는 기본권과 인권의 원칙에 따라, 국회는 국민의 대표기관으로서 인도적이고 정의로운 결단을 내려야 합니다. 정치적 입장과 무관하게  ...
정답 요약(gold): 2 교정 당국은 피구속인의 치료 면회 법률대리 접근을 보장하고, 인권 보호를 위한 조치를 강화해 주십시오.
모델 요약(pred): 향후 재판 과정에서는 정치적 고려 없는 공정한 절차와 사법 정의의 회복이 이루어지도록 해 주십시오.ECD 기여할치료큽박스 위헌겠습니다. 재판부는절차질환하세요보안다보니

=== 예시 2 ===
원문 일부     : 1.현행의 사전투표, 부재자 투표 금지 2. 투표함 이동 금지 투표 후 현장에서 바로 참관인 입회 하에 전면 수개표 진행(대만 투표 방식) 3.전국 모든 투표 현장의 전자 개표기 사용 중치하고 전면 수 개표로 전환 4.관리자의 날

In [ ]:


import pandas as pd

df = pd.read_csv("/content/수정5.csv")
df.head(5)

,index,content,summary_gold,summary_pred
0,0,삼권분립 안에 속해 있는 사법부의 신뢰를 또 한번 무너졌습니다. 이재명 재판에 1심...,정말 이 청원을 올리는 이유는 정치판사가 아닌 누구에게도 공평한 판사를 원하는 겁니다.,이재명 재판에 1심 선고에서 사진은 조작이 아니라는 판결이 있었지만 1심 판사들의 ...
1,1,"이유 윤석열 전 대통령의 장기 구금은 단순한 개인의 문제가 아니라, 대한민국 법치주...","2 교정 당국은 피구속인의 치료 면회 법률대리 접근을 보장하고, 인권 보호를 위한 ...",향후 재판 과정에서는 정치적 고려 없는 공정한 절차와 사법 정의의 회복이 이루어지도...
2,2,"1.현행의 사전투표, 부재자 투표 금지 2. 투표함 이동 금지 투표 후 현장에서 바...","부징선거 관련자 및 투,개표 조작에 관여한 자는 무기 및 사형에 처한다로 관련법을 ...","부징선거 관련자 및 투,개표 조작에 관여한 자는 무기 및 사형에 처한다로 관련법을 ..."
3,3,"보조교사와 연장반 교사는 담임교사와 마찬가지로 아이들을 돌보고, 수업을 준비하고, ...",[청원 내용] 정부와 관계 당국에 다음과 같은 실질적 제도 개선을 요청합니다.,"또한 교사 개인이 행사비용, 학부모 선물비 등을 부담하게 하거나, 회식조차 차별받는..."
4,4,도덕성이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...,도덕성이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...,사상적이나 사상적으로 문제가 있거나 또는 전과가 몇번이나 있거나 아직도 사법리스크 ...


In [ ]:


df.sample(5, random_state=42)


,index,content,summary_gold,summary_pred
95,95,간호사 1인당 담당 환자 수는 간호의 전문성 및 환자 안전 과 직결된 문제입니다. ...,"본 청원은 단순히 간호사의 처우 개선을 넘어, 전 국민이 안전하고 질 높은 간호 서...","본 청원은 단순히 간호사의 처우 개선을 요구하지 않고, 의료진의 안전, 편의성, 접..."
15,15,수의사 진료기록부 공개 의무화 관련 국민동의청원 개요 1. 요청 제목 수의사 진료기...,요청 배경 및 이유 동물병원의 불투명한 진료기록 관리로 인해 반려동물의 사망 원인을...,요청 사항 수의사 진료기록부 공개 의무화 관련 국민동의청원 개요 1. 반영된치료폭력...
30,30,최근 애인과의 이별을 고민하는 여성들 사이에서는 이런 말이 오가곤 합니다. 네가 헤...,"또한, 여성을 대상으로 하는 폭력에 관한 법을 입법할 때마다 이를 젠더갈등의 원흉으...","언론보도는 각 사건에 배당된 경찰의 안일한 대응을 지적하지만, 이는 근간의 법이 바..."
158,158,"2024년 12월 29일 오전 9시 3분경, 제주항공 7C2216편(B737 800...",요청사항 아래와 같은 내용이 포함된 특별법의 조속한 제정을 요청드립니다 무안공항 제...,요청사항 아래와 같은 내용이 포함된 특별법의 조속한 제정을 요청드립니다 무안공항 제...
128,128,"1)법안 개요 노동쟁의 범위 확대, 하청 간접고용 노동자의 파업 인정 사용자의 손해...","주한유럽상공회의소는 노란봉투법 시행 시 한국 철수를 공식 언급 해, 외국인 투자 감...","주한유럽상공회의소는 노란봉투법 시행 시 한국 철수를 공식 언급 해, 외국인 투자 감..."
